In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 8)
!date

%load_ext autoreload
%autoreload 2

Thu Jun  6 14:06:44 PDT 2019


# 1940 Census Data for testing DP

https://usa.ipums.org/usa/1940CensusDASTestData.shtml

https://www.ipums.org/census-bureau-data.shtml

Loading all of the census into main memory appears to require more than 256G on the IHME Cluster.

In [10]:
colspec_str = """  RECTYPE            H   1              1      X 
  YEAR               H   2-5            4      X 
  DATANUM            H   6-7            2      X 
  SERIAL             H   8-15           8      X 
  NUMPREC            H  16-17           2      X 
  SUBSAMP            H  18-19           2      X 
  HHWT               H  20-29          10      X 
  NUMPERHH           H  30-33           4      X 
  HHTYPE             H  34              1      X 
  DWELLING           H  35-42           8      X 
  SLPERNUM           H  43-44           2      X 
  CPI99              H  45-49           5      X 
  REGION             H  50-51           2      X 
  STATEICP           H  52-53           2      X 
  STATEFIP           H  54-55           2      X 
  COUNTY             H  56-59           4      X 
  URBAN              H  60              1      X 
  METRO              H  61              1      X 
  METAREA            H  62-64           3      X 
  METAREAD           H  65-68           4      X 
  CITY               H  69-72           4      X 
  CITYPOP            H  73-77           5      X 
  SIZEPL             H  78-79           2      X 
  URBPOP             H  80-84           5      X 
  SEA                H  85-87           3      X 
  WARD               H  88-90           3      X 
  CNTRY              H  91-93           3      X 
  GQ                 H  94              1      X 
  GQTYPE             H  95              1      X 
  GQTYPED            H  96-98           3      X 
  GQFUNDS            H  99-100          2      X 
  FARM               H 101              1      X 
  OWNERSHP           H 102              1      X 
  OWNERSHPD          H 103-104          2      X 
  RENT               H 105-108          4      X 
  VALUEH             H 109-115          7      X 
  NFAMS              H 116-117          2      X 
  NSUBFAM            H 118              1      X 
  NCOUPLES           H 119              1      X 
  NMOTHERS           H 120              1      X 
  NFATHERS           H 121              1      X 
  MULTGEN            H 122              1      X 
  MULTGEND           H 123-124          2      X 
  ENUMDIST           H 125-128          4      X 
  SUPDIST            H 129-131          3      X 
  RESPOND            H 132              1      X 
  SPLIT              H 133              1      X 
  SPLITHID           H 134-141          8      X 
  SPLITNUM           H 142-145          4      X 
  SPLIT40            H 146              1      X 
  SERIAL40           H 147-154          8      X 
  NUMPREC40          H 155-158          4      X 
  EDMISS             H 159              1      X 
  RECTYPEP           P   1              1      X 
  YEARP              P   2-5            4      X 
  DATANUMP           P   6-7            2      X 
  SERIALP            P   8-15           8      X 
  PERNUM             P  16-19           4      X 
  PERWT              P  20-29          10      X 
  SLWT               P  30-39          10      X 
  SLREC              P  40              1      X 
  RESPONDT           P  41              1      X 
  FAMUNIT            P  42-43           2      X 
  FAMSIZE            P  44-45           2      X 
  SUBFAM             P  46              1      X 
  SFTYPE             P  47              1      X 
  SFRELATE           P  48              1      X 
  MOMLOC             P  49-50           2      X 
  STEPMOM            P  51              1      X 
  MOMRULE_HIST       P  52              1      X 
  POPLOC             P  53-54           2      X 
  STEPPOP            P  55              1      X 
  POPRULE_HIST       P  56              1      X 
  SPLOC              P  57-58           2      X 
  SPRULE_HIST        P  59              1      X 
  NCHILD             P  60              1      X 
  NCHLT5             P  61              1      X 
  NSIBS              P  62              1      X 
  ELDCH              P  63-64           2      X 
  YNGCH              P  65-66           2      X 
  RELATE             P  67-68           2      X 
  RELATED            P  69-72           4      X 
  SEX                P  73              1      X 
  AGE                P  74-76           3      X 
  AGEMONTH           P  77-78           2      X 
  MARST              P  79              1      X 
  MARRNO             P  80              1      X 
  AGEMARR            P  81-82           2      X 
  CHBORN             P  83-84           2      X 
  RACE               P  85              1      X 
  RACED              P  86-88           3      X 
  HISPAN             P  89              1      X 
  HISPAND            P  90-92           3      X 
  BPL                P  93-95           3      X 
  BPLD               P  96-100          5      X 
  MBPL               P 101-103          3      X 
  MBPLD              P 104-108          5      X 
  FBPL               P 109-111          3      X 
  FBPLD              P 112-116          5      X 
  NATIVITY           P 117              1      X 
  CITIZEN            P 118              1      X 
  MTONGUE            P 119-120          2      X 
  MTONGUED           P 121-124          4      X 
  SPANNAME           P 125              1      X 
  HISPRULE           P 126              1      X 
  SCHOOL             P 127              1      X 
  HIGRADE            P 128-129          2      X 
  HIGRADED           P 130-132          3      X 
  EDUC               P 133-134          2      X 
  EDUCD              P 135-137          3      X 
  EMPSTAT            P 138              1      X 
  EMPSTATD           P 139-140          2      X 
  LABFORCE           P 141              1      X 
  OCC                P 142-145          4      X 
  OCC1950            P 146-148          3      X 
  IND                P 149-152          4      X 
  IND1950            P 153-155          3      X 
  CLASSWKR           P 156              1      X 
  CLASSWKRD          P 157-158          2      X 
  WKSWORK1           P 159-160          2      X 
  WKSWORK2           P 161              1      X 
  HRSWORK1           P 162-163          2      X 
  HRSWORK2           P 164              1      X 
  DURUNEMP           P 165-167          3      X 
  UOCC               P 168-170          3      X 
  UOCC95             P 171-173          3      X 
  UIND               P 174-176          3      X 
  UCLASSWK           P 177              1      X 
  INCWAGE            P 178-183          6      X 
  INCNONWG           P 184              1      X 
  OCCSCORE           P 185-186          2      X 
  SEI                P 187-188          2      X 
  PRESGL             P 189-191          3      X 
  ERSCOR50           P 192-195          4      X 
  EDSCOR50           P 196-199          4      X 
  NPBOSS50           P 200-203          4      X 
  MIGRATE5           P 204              1      X 
  MIGRATE5D          P 205-206          2      X 
  MIGPLAC5           P 207-209          3      X 
  MIGMET5            P 210-213          4      X 
  MIGTYPE5           P 214              1      X 
  MIGCITY5           P 215-218          4      X 
  MIGSEA5            P 219-221          3      X 
  SAMEPLAC           P 222              1      X 
  SAMESEA5           P 223              1      X 
  MIGCOUNTY          P 224-227          4      X 
  VETSTAT            P 228              1      X 
  VETSTATD           P 229-230          2      X 
  VET1940            P 231              1      X 
  VETWWI             P 232              1      X 
  VETPER             P 233              1      X 
  VETCHILD           P 234              1      X 
  HISTID             P 235-270         36      X 
  SURSIM             P 271-272          2      X 
  SSENROLL           P 273              1      X """

In [12]:
with open('data/full_ipums_1940_census_colspecs.txt', 'w') as f:
    f.write(colspec_str)

In [3]:
colspecs = pd.Series()
for line in colspec_str.split('\n'):
    name = line.split()[0].strip()
    val_str = line.split()[2]
    
    start_str = val_str.split('-')[0]
    start_index = int(start_str) - 1

    end_str = val_str.split('-')[-1]
    end_index = int(end_str)
    
#     print(name, val_str, (start_index, end_index))
    colspecs[name] = (start_index, end_index)

In [6]:
pd.DataFrame([colspecs]).to_csv('data/ipums_1940_census_colspecs.csv')

In [ ]:
%%time

fname = '/home/j/Project/Models/us_census/EXT1940USCB.dat.gz'

df = pd.read_fwf(fname, list(colspecs), names=colspecs.index, #nrows=10_000,
                 compression='gzip')
df

In [ ]:
df.shape

In [ ]:
# qlogin -l fthread=1 -l m_mem_free=100G -P ihme_general -now no -q long.q -l archive=TRUE

In [52]:
from dask_jobqueue import SGECluster

cluster = SGECluster(queue='all.q',
                     project='ihme_general',
                     resource_spec='m_mem_free=20G',  # for memory requests, this also needs to be specified
                     job_extralist='-l fthread=2',
                     processes=2,   # we request 10 processes per worker
                     memory='20GB',  # for memory requests, this must be specified
                     cores=2, # cores different than processes?
                     )

In [53]:
from dask.distributed import Client
client = Client(cluster)  # Connect this local process to remote workers

In [54]:
client

Client Scheduler: tcp://10.158.100.169:26758,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [55]:
cluster.scale(32)

tornado.application - ERROR - Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x2b4d367838c8>, 32)
Traceback (most recent call last):
  File "/share/code/abie/miniconda3/lib/python3.6/site-packages/tornado/ioloop.py", line 758, in _run_callback
    ret = callback()
  File "/share/code/abie/miniconda3/lib/python3.6/site-packages/tornado/stack_context.py", line 300, in null_wrapper
    return fn(*args, **kwargs)
  File "/share/code/abie/miniconda3/lib/python3.6/site-packages/dask_jobqueue/core.py", line 416, in scale_up
    self.start_workers(n - self._count_active_and_pending_workers())
  File "/share/code/abie/miniconda3/lib/python3.6/site-packages/dask_jobqueue/core.py", line 330, in start_workers
    out = self._submit_job(fn)
  File "/share/code/abie/miniconda3/lib/python3.6/site-packages/dask_jobqueue/core.py", line 322, in _submit_job
    return self._call(shlex.split(self.submit_command) + [script_filename])
  File "/share/code/abie/miniconda3/lib/

In [51]:
!cat /tmp/tmpodroe78z.sh

#!/bin/bash

#!/usr/bin/env bash
#$ -N dask-worker
#$ -q all.q
#$ -P ihme_general
#$ -l m_mem_free=20G
#$ -l h_rt=00:30:00
#$ -cwd
#$ -j y



/share/code/abie/miniconda3/bin/python -m distributed.cli.dask_worker tcp://10.158.100.169:16307 --nthreads 1 --nprocs 2 --memory-limit 10.00GB --name dask-worker--${JOB_ID}-- --death-timeout 60


In [29]:
cluster